# Holt Winters Experiments

___
___

**Dreifaches Exponential Smoothing**

Komponenten:
1. Level - Alpha
2. Trend - Beta
3. Season - Gamma
<br/><br/>
___

## #1 Read Data

In [2]:
import numpy as np

# Read the data
data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/energy_generation_solar_small.csv', delimiter=',', skip_header=1)

print(data.shape)

initial_data = data

(35045,)


In [1]:
import numpy as np

# Read the data
data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/berlin_temperature_data_small.csv', delimiter=',', skip_header=1)

print(data.shape)

initial_data_temperature = data

(13254,)


___
## #2 Error Metrics

In [3]:
import numpy as np

def sum_of_squares_error(original, prediction):
    """
    Calculates the sum of squares error between two vectors.

    Parameters:
    original (numpy.ndarray): The original vector.
    prediction (numpy.ndarray): The prediction vector.

    Returns:
    float: The sum of squares error.
    """
    return np.sum((original - prediction) ** 2)

# Example:
original = np.array([1, 2, 3])
prediction = np.array([3, 4, 5])
error = sum_of_squares_error(original, prediction)
print(f"Sum of Squares Error: {error}")


Sum of Squares Error: 12


___
## #3 Holt Winters 

### Basic Holt Winters

In [7]:
# Basic Holt Winters as a function

import numpy as np

def triple_exponential_smoothing_basic(data, alpha, beta, gamma, season_length):
    """
    Performs triple exponential smoothing on the given data.

    Args:
        data: The input data as a NumPy array.
        alpha: Smoothing factor for level.
        beta: Smoothing factor for trend.
        gamma: Smoothing factor for seasonality.
        season_length: Length of the seasonal period.

    Returns:
        A tuple containing the smoothed values, level, trend, and season components.
    """

    n = data.shape[0]

    # Initialize arrays
    level = np.zeros(n)
    trend = np.zeros(n)
    season = np.zeros(n)
    smoothed = np.zeros(n)

    # Initial values
    level[0] = data[0]
    trend[0] = data[1] - data[0]
    season[:season_length] = data[:season_length] - level[0]

    if season_length == 0:
        season_length = n

    # Calculate level, trend, and season components
    for t in range(1, n):
        level[t] = alpha * (data[t] - season[t - season_length]) + (1 - alpha) * (level[t-1] + trend[t-1])
        trend[t] = beta * (level[t] - level[t-1]) + (1 - beta) * trend[t-1]
        if t >= season_length:
            season[t] = gamma * (data[t] - level[t]) + (1 - gamma) * season[t - season_length]
        smoothed[t] = level[t] + trend[t] + season[t-t%season_length]
        # Print intermediate values for debugging
        # print(f"t={t}, level={level[t]}, trend={trend[t]}, season={season[t]}, smoothed={smoothed[t]}")
    # Return results
    return smoothed

# Example:
data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/berlin_temperature_data_small.csv', delimiter=',', skip_header=1)
smoothed = triple_exponential_smoothing_basic(data, 0.2, 0.1, 0.3, 365)
last_smoothed_value = smoothed[-1]
print("Holt Winters Exponential Smoothing:")
print("The last smoothed value for energy_generation_solar is:", last_smoothed_value)
#print(smoothed)

Holt Winters Exponential Smoothing:
The last smoothed value for energy_generation_solar is: 3.9374900912503508


In [20]:
# Basic HW ES
# Working out the formula into code

import numpy as np
import time

# Read data 
data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/energy_generation_solar_small.csv', delimiter=',', skip_header=1)

# List to store execution times
number_of_executions = 10
execution_times = []

# Smoothing factors
alpha = 0.2
beta = 0.1
gamma = 0.3
season_length = 24  # Season = 1 Day for 1 value per hour

# Holt-Winters
for i in range(number_of_executions):

    # Initialization or reset of parameter arrays
    n = len(data)
    level = np.zeros(n)
    trend = np.zeros(n)
    season = np.zeros(n)
    smoothed = np.zeros(n)

    # Assign initial values
    level[0] = data[0]
    trend[0] = data[1] - data[0]
    season[:season_length] = data[:season_length] - level[0]

    # Start teh timer
    start_time = time.time()

    # Holt-Winters
    for t in range(1, n):

        # Level: a[t] = α * (y[t] − s[t−j]) + (1 − α) * (a[t−1] + b[t−1])   
        level[t] = alpha * (data[t] - season[t-season_length]) + (1 - alpha) * (level[t-1] + trend[t-1])

        # Trend: b[t] = β * (a[t] − a[t−1]) + (1 − β) * b[t−1]
        trend[t] = beta * (level[t] - level[t-1]) + (1 - beta) * trend[t-1]

        # Season: s[t] = γ * (y[t] − a[t]) + (1 − γ) * s[t−j]
        if t >= season_length:
            season[t] = gamma * (data[t] - level[t]) + (1 - gamma) * season[t-season_length]

        # Forecast: Y[t,h] = a[t] + b[t] + s[t−j+m]
        smoothed[t] = level[t] + trend[t] + season[t-t%season_length]

        # Print intermediate values for debugging
        #print(f"t={t}, level={level[t]}, trend={trend[t]}, season={season[t]}, smoothed={smoothed[t]}")
      
    # Stop the timer
    end_time = time.time()
    function_time = end_time - start_time
    execution_times.append(function_time)
    #print(function_time)

# Calculate the function time
function_time = np.mean(execution_times)

# Print results
print("\n### Dataset energy_generation_solar")
print("### Holt-Winters\n")
#print("The smoothed values for energy_generation_solar are:", smoothed)

# Print last smoothed value
smoothed_value = smoothed[-1]
print("The last smoothed value for energy_generation_solar is:", smoothed_value)
print(f"The function was executed in {function_time} seconds")


### Dataset energy_generation_solar
### Holt-Winters

The last smoothed value for energy_generation_solar is: 90.60914407238147
The function was executed in 0.0939183235168457 seconds


In [8]:
# Vectorized

# read data
data = initial_data

# Smoothing factors
alpha = 0.2
beta = 0.1
gamma = 0.3
season_length = 24  # 1 Day for 1 value per hour

# List to store execution times
number_of_executions = 1
execution_times = []

# Perform exponential smoothing on the data
for i in range(number_of_executions):
    start_time = time.time()

    # Perform exponential smoothing on the data
    n = data.shape[0]

    # Level calculation
    level_weights = alpha * (1 - alpha) ** np.arange(n)
    level_smoothed = np.cumsum(level_weights * data[::-1])[::-1]
    level = level_smoothed[0] / level_weights.sum()

    # Trend calculation
    trend_weights = beta * (1 - beta) ** np.arange(n)
    trend_smoothed = np.cumsum(trend_weights * (level_smoothed - np.roll(level_smoothed, 1))[::-1])[::-1]
    trend = trend_smoothed[0] / trend_weights.sum()

    # Season calculation
    season_weights = gamma * (1 - gamma) ** np.arange(season_length)
    season_smoothed = np.cumsum(season_weights * (data[:season_length] - level_smoothed[:season_length])[::-1])[::-1]
    season = season_smoothed[0] / season_weights.sum()

    # Forecast: Y[t,h] = a[t] + b[t] + s[t−j+m]
    smoothed = level + trend + season

    # Stop the timer
    end_time = time.time()
    execution_times.append(end_time - start_time)

# Calculate the function time
function_time = np.median(execution_times)

# Print the results
print("### Dataset wind_speed")
print("### #2 Vectorized \n")
print("The last smoothed value for wind_speed is:", smoothed)
print(f"The function was executed in {function_time} seconds")

### Dataset wind_speed
### #2 Vectorized 

The last smoothed value for wind_speed is: 267.5670779769958
The function was executed in 0.02587127685546875 seconds


In [9]:
import numpy as np
import time

# Read data
data = initial_data

# Smoothing factors
alpha = 0.2
beta = 0.1
gamma = 0.3
season_length = 24  # 1 Day for 1 value per hour

# Number of executions for timing
number_of_executions = 1
execution_times = []

# Perform exponential smoothing on the data
for i in range(number_of_executions):
    start_time = time.time()

    n = data.shape[0]

    # Initialize arrays
    level = np.zeros(n)
    trend = np.zeros(n)
    season = np.zeros(n)
    smoothed = np.zeros(n)

    # Initial values
    level[0] = data[0]
    trend[0] = data[1] - data[0]
    season[:season_length] = data[:season_length] - level[0]

    # Vectorized calculation of level, trend, and season components
    indices = np.arange(n)
    season_indices = np.mod(indices - season_length, n)

    level[1:] = alpha * (data[1:] - season[season_indices[1:]]) + (1 - alpha) * (level[:-1] + trend[:-1])
    trend[1:] = beta * (level[1:] - level[:-1]) + (1 - beta) * trend[:-1]
    season[season_length:] = gamma * (data[season_length:] - level[season_length:]) + (1 - gamma) * season[:-season_length]

    smoothed = level + trend + season[season_indices]

    # Stop the timer
    end_time = time.time()
    execution_times.append(end_time - start_time)

# Calculate the function time
function_time = np.median(execution_times)

# Print the results
print("### Dataset wind_speed")
print("### #2 Vectorized \n")
print("The last smoothed value for wind_speed is:", smoothed[-1])
print(f"The function was executed in {function_time} seconds")


### Dataset wind_speed
### #2 Vectorized 

The last smoothed value for wind_speed is: 13.399999999999999
The function was executed in 0.0032880306243896484 seconds


___
### Vectorized Holt Winters

In [21]:
import numpy as np
import time

def holt_winters_smoothing(data, alpha, beta, gamma, season_length):
    n = data.shape[0]

    # Initialize arrays
    level = np.zeros(n)
    trend = np.zeros(n)
    season = np.zeros(season_length)
    smoothed = np.zeros(n)

    # Initial values
    level[0] = data[0]
    trend[0] = data[1] - data[0]
    season[:season_length] = data[:season_length] - level[0]

    # Vectorized approach for each season
    for season_start in range(0, n, season_length):
        season_end = min(season_start + season_length, n)
        season_indices = np.arange(season_start, season_end)
        
        # Calculate level
        level_weights = alpha * (1 - alpha) ** np.arange(season_end - season_start)
        level_smoothed = np.cumsum(level_weights * data[season_indices][::-1])[::-1]
        level[season_indices] = level_smoothed / level_weights.sum()

        # Calculate trend
        trend_weights = beta * (1 - beta) ** np.arange(season_end - season_start)
        trend_smoothed = np.cumsum(trend_weights * (level_smoothed - np.roll(level_smoothed, 1))[::-1])[::-1]
        trend[season_indices] = trend_smoothed / trend_weights.sum()

        # Calculate season
        season_weights = gamma * (1 - gamma) ** np.arange(season_end - season_start)
        season_smoothed = np.cumsum(season_weights * (data[season_indices] - level_smoothed)[::-1])[::-1]
        season[season_indices % season_length] = season_smoothed / season_weights.sum()

        # Combine components
        smoothed[season_indices] = level[season_indices] + trend[season_indices] + season[season_indices % season_length]

    return smoothed

# Example usage
data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/energy_generation_solar_small.csv', delimiter=',', skip_header=1)
alpha = 0.2
beta = 0.1
gamma = 0.3
season_length = 24

start_time = time.time()
smoothed = holt_winters_smoothing(data, alpha, beta, gamma, season_length)
end_time = time.time()

# Print the results
print("### Dataset wind_speed")
print("### #2 Vectorized \n")
print("The last smoothed value for wind_speed is:", smoothed[-1])
print(f"The function was executed in {end_time - start_time} seconds")
#print(smoothed)

### Dataset wind_speed
### #2 Vectorized 

The last smoothed value for wind_speed is: 16.953655622884888
The function was executed in 0.08023500442504883 seconds


In [26]:
import numpy as np
import time

def holt_winters_smoothing_v8(data, alpha, beta, gamma, season_length):
    n = data.shape[0]

    # Initialize arrays
    level = np.zeros(n)
    trend = np.zeros(n)
    season = np.zeros(season_length)
    smoothed = np.zeros(n)

    # Initial values
    level[0] = data[0]
    trend[0] = data[1] - data[0]
    season[:season_length] = data[:season_length] - level[0]

    # Vectorized approach for each season
    for season_start in range(0, n, season_length):
        season_end = min(season_start + season_length, n)
        season_indices = np.arange(season_start, season_end)
        
        # Calculate level
        level_weights = alpha * (1 - alpha) ** np.arange(season_end - season_start)
        level_smoothed = np.cumsum(level_weights * data[season_indices][::-1])[::-1]
        level[season_indices] = level_smoothed / level_weights.sum()

        # Calculate trend
        trend_weights = beta * (1 - beta) ** np.arange(season_end - season_start)
        trend_smoothed = np.cumsum(trend_weights * (level_smoothed - np.roll(level_smoothed, 1))[::-1])[::-1]
        trend[season_indices] = trend_smoothed / trend_weights.sum()

        # Calculate season
        if season_start == 0:
            season_weights = np.zeros(season_end - season_start)
            season_smoothed = np.zeros(season_end - season_start)
        else:
            season_weights = gamma * (1 - gamma) ** np.arange(season_end - season_start)
            season_smoothed = np.cumsum(season_weights * (data[season_indices] - level_smoothed)[::-1])[::-1]
            season[season_indices % season_length] = season_smoothed / season_weights.sum()

        smoothed[season_indices] = level[season_indices] + trend[season_indices] + season[season_indices % season_length]

        # Print intermediate values for debugging
        #print(f"Season start: {season_start}, Season end: {season_end}")
        #print(f"Level: {level[season_indices]}")
        #print(f"Trend: {trend[season_indices]}")
        #print(f"Season: {season[season_indices % season_length]}")
        #print(f"Smoothed: {smoothed[season_indices]}")

    return smoothed

# Example usage
data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/energy_generation_solar_small.csv', delimiter=',', skip_header=1)
alpha = 0.2
beta = 0.1
gamma = 0.3
season_length = 24

start_time = time.time()
smoothed = holt_winters_smoothing_v8(data, alpha, beta, gamma, season_length)
end_time = time.time()

# Print the results
print("### Dataset wind_speed")
print("### #2 Vectorized \n")
print("The last smoothed value for wind_speed is:", smoothed[-1])
print(f"The function was executed in {end_time - start_time} seconds")

### Dataset wind_speed
### #2 Vectorized 

The last smoothed value for wind_speed is: 31.71308274776318
The function was executed in 0.0765988826751709 seconds


In [20]:
import numpy as np
import time

def holt_winters_smoothing_v2(data, alpha, beta, gamma, season_length):
    n = data.shape[0]

    # Initialize arrays
    level = np.zeros(n)
    trend = np.zeros(n)
    season = np.zeros(season_length)
    smoothed = np.zeros(n)

    # Initial values
    level[0] = data[0]
    trend[0] = data[1] - data[0]
    season[:season_length] = data[:season_length] - level[0]

    # Vectorized approach for each season
    for season_start in range(0, n, season_length):
        season_end = min(season_start + season_length, n)
        season_indices = np.arange(season_start, season_end)
        
        # Calculate level
        level_weights = alpha * (1 - alpha) ** np.arange(season_end - season_start)
        level_smoothed = np.cumsum(level_weights * data[season_indices][::-1])[::-1]
        level[season_indices] = level_smoothed / level_weights.sum()

        # Calculate trend
        trend_weights = beta * (1 - beta) ** np.arange(season_end - season_start)
        trend_smoothed = np.cumsum(trend_weights * (level_smoothed - np.roll(level_smoothed, 1))[::-1])[::-1]
        trend[season_indices] = trend_smoothed / trend_weights.sum()

        # Calculate season
        season_weights = gamma * (1 - gamma) ** np.arange(season_end - season_start)
        season_smoothed = np.cumsum(season_weights * (data[season_indices] - level_smoothed)[::-1])[::-1]
        season[season_indices % season_length] = season_smoothed / season_weights.sum()

        # Combine components
        smoothed[season_indices] = level[season_indices] + trend[season_indices] + season[season_indices % season_length]

    return smoothed

# Example usage
data = np.genfromtxt('/Users/niklas/Documents/GitHub/Uni/10_Masterarbeit/data_files/prepocessed_datafiles/energy_generation_solar_small.csv', delimiter=',', skip_header=1)
alpha = 0.2
beta = 0.1
gamma = 0.3
season_length = 24

start_time = time.time()
smoothed = holt_winters_smoothing_v2(data, alpha, beta, gamma, season_length)
end_time = time.time()

# Print the results
print("### Dataset wind_speed")
print("### Vectorized \n")
print("The last smoothed value for wind_speed is:", smoothed[-1])
print(f"The function was executed in {end_time - start_time} seconds")


### Dataset wind_speed
### Vectorized 

The last smoothed value for wind_speed is: 16.953655622884888
The function was executed in 0.12026500701904297 seconds


In [19]:
import numpy as np
import time

def holt_winters_smoothing_v3(data, alpha, beta, gamma, season_length):
    n = data.shape[0]

    # Initialize arrays
    level = np.zeros(n)
    trend = np.zeros(n)
    season = np.zeros(season_length)
    smoothed = np.zeros(n)

    # Initial values
    level[0] = data[0]
    trend[0] = data[1] - data[0]
    season[:season_length] = data[:season_length] - level[0]

    # Vectorized approach for each season
    for season_start in range(0, n, season_length):
        season_end = min(season_start + season_length, n)
        season_indices = np.arange(season_start, season_end)
        
        # Calculate level
        level_weights = alpha * (1 - alpha) ** np.arange(season_end - season_start)
        level_smoothed = np.cumsum(level_weights * data[season_indices][::-1])[::-1]
        level[season_indices] = level_smoothed / level_weights.sum()

        # Calculate trend
        trend_weights = beta * (1 - beta) ** np.arange(season_end - season_start)
        trend_smoothed = np.cumsum(trend_weights * (level_smoothed - np.roll(level_smoothed, 1))[::-1])[::-1]
        trend[season_indices] = trend_smoothed / trend_weights.sum()

        # Calculate season
        season_weights = gamma * (1 - gamma) ** np.arange(season_end - season_start)
        season_smoothed = np.cumsum(season_weights * (data[season_indices] - level_smoothed)[::-1])[::-1]
        season[season_indices % season_length] = season_smoothed / season_weights.sum()

        # Combine components
        smoothed[season_indices] = level[season_indices] + trend[season_indices] + season[season_indices % season_length]

    return smoothed

# Example usage
data = np.genfromtxt('/Users/niklas/Documents/GitHub/Uni/10_Masterarbeit/data_files/prepocessed_datafiles/energy_generation_solar_small.csv', delimiter=',', skip_header=1)
alpha = 0.2
beta = 0.1
gamma = 0.3
season_length = 24

start_time = time.time()
smoothed = holt_winters_smoothing_v3(data, alpha, beta, gamma, season_length)
end_time = time.time()

# Print the results
print("### Dataset wind_speed")
print("### #2 Vectorized \n")
print("The last smoothed value for wind_speed is:", smoothed[-1])
print(f"The function was executed in {end_time - start_time} seconds")
print(smoothed)

### Dataset wind_speed
### #2 Vectorized 

The last smoothed value for wind_speed is: 16.953655622884888
The function was executed in 0.12407493591308594 seconds
[838.3959636  832.61424098 832.60639426 ...  36.50524022  28.46714198
  16.95365562]


In [31]:
import numpy as np
import time

def holt_winters_smoothing_v4(data, alpha, beta, gamma, season_length):
    n = data.shape[0]

    # Initialize arrays
    level = np.zeros(n)
    trend = np.zeros(n)
    season = np.zeros(season_length)
    smoothed = np.zeros(n)

    # Initial values
    level[0] = data[0]
    trend[0] = data[1] - data[0]
    season[:season_length] = data[:season_length] - level[0]

    # Create weight matrices
    level_weights = alpha * (1 - alpha) ** np.arange(season_length)
    trend_weights = beta * (1 - beta) ** np.arange(season_length)
    season_weights = gamma * (1 - gamma) ** np.arange(season_length)

    # Vectorized approach for each season
    for season_start in range(0, n, season_length):
        season_end = min(season_start + season_length, n)
        season_indices = np.arange(season_start, season_end)
        
        # Calculate level
        level_smoothed = np.cumsum(level_weights[:season_end - season_start] * data[season_indices][::-1])[::-1]
        level[season_indices] = level_smoothed / level_weights[:season_end - season_start].sum()

        # Calculate trend
        trend_smoothed = np.cumsum(trend_weights[:season_end - season_start] * (level_smoothed - np.roll(level_smoothed, 1))[::-1])[::-1]
        trend[season_indices] = trend_smoothed / trend_weights[:season_end - season_start].sum()

        # Calculate season
        season_smoothed = np.cumsum(season_weights[:season_end - season_start] * (data[season_indices] - level_smoothed)[::-1])[::-1]
        season[season_indices % season_length] = season_smoothed / season_weights[:season_end - season_start].sum()

        # Combine components
        smoothed[season_indices] = level[season_indices] + trend[season_indices] + season[season_indices % season_length]

    return smoothed

# Example usage
data = np.genfromtxt('/Users/niklas/Documents/GitHub/Uni/10_Masterarbeit/data_files/prepocessed_datafiles/energy_generation_solar_small.csv', delimiter=',', skip_header=1)
alpha = 0.2
beta = 0.1
gamma = 0.3
season_length = 24

start_time = time.time()
smoothed = holt_winters_smoothing_v4(data, alpha, beta, gamma, season_length)
end_time = time.time()

# Print the results
print("### Dataset wind_speed")
print("### Vectorized \n")
print("The last smoothed value for wind_speed is:", smoothed[-1])
print(f"The function was executed in {end_time - start_time} seconds")
print(smoothed)

### Dataset wind_speed
### Vectorized 

The last smoothed value for wind_speed is: 16.953655622884888
The function was executed in 0.09415125846862793 seconds
[838.3959636  832.61424098 832.60639426 ...  36.50524022  28.46714198
  16.95365562]


___
## #4 Experiments

In [14]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Sample data

data = np.genfromtxt('/Users/niklas/Documents/GitHub/MasterThesis/0_Data_files/preprocessed_data_files/berlin_temperature_data_small.csv', delimiter=',', skip_header=1)

# Create a pandas Series
data_series = pd.Series(data)

# Apply Holt-Winters Exponential Smoothing
model = ExponentialSmoothing(data_series, trend='add', seasonal='add', seasonal_periods=365)
fit = model.fit()

# Forecast
forecast = fit.forecast(1)
print(forecast)
last_smoothed_value = forecast.iloc[-1]
print("Holt Winters Exponential Smoothing:")
print("The last smoothed value for energy_generation_solar is:", last_smoothed_value)


/Users/niklas/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/holtwinters/model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


12608    16.533619
dtype: float64
Holt Winters Exponential Smoothing:
The last smoothed value for energy_generation_solar is: 16.53361914225591


In [35]:
import numpy as np
import time

# Read data 
data = initial_data

# List to store execution times
number_of_executions = 10
execution_times = []

# Smoothing factors
alpha = 0.2
beta = 0.1
gamma = 0.3
season_length = 24  # Season = 1 Day for 1 value per hour

# Holt-Winters
for i in range(number_of_executions):

    # Start teh timer
    start_time = time.time()

    # Holt-Winters
    triple_exponential_smoothing_basic(data, alpha, beta, gamma, season_length)

    # Stop the timer
    end_time = time.time()
    function_time = end_time - start_time
    execution_times.append(function_time)
    #print(function_time)

# Calculate the function time
function_time = np.mean(execution_times)

# Print results
print("\n### Dataset energy_generation_solar")
print("### Holt-Winters\n")
print("The last smoothed value for energy_generation_solar is:", smoothed_value)
print(f"The function was executed in {function_time} seconds")


### Dataset energy_generation_solar
### Holt-Winters

The last smoothed value for energy_generation_solar is: 56.202351904167585
The function was executed in 0.08267958164215088 seconds


In [37]:
import numpy as np
import time

# Read data 
data = initial_data

# List to store execution times
number_of_executions = 10
execution_times = []

# Smoothing factors
alpha = 0.7
beta = 0
gamma = 0
season_length = 0  # Season = 1 Day for 1 value per hour

# Holt-Winters
for i in range(number_of_executions):

    # Start teh timer
    start_time = time.time()

    # Holt-Winters
    triple_exponential_smoothing_basic(data, alpha, beta, gamma, season_length)

    # Stop the timer
    end_time = time.time()
    function_time = end_time - start_time
    execution_times.append(function_time)
    #print(function_time)

# Calculate the function time
function_time = np.mean(execution_times)

# Print results
print("\n### Dataset energy_generation_solar")
print("### Holt-Winters\n")
print("The last smoothed value for energy_generation_solar is:", smoothed_value)
print(f"The function was executed in {function_time} seconds")


### Dataset energy_generation_solar
### Holt-Winters

The last smoothed value for energy_generation_solar is: 56.202351904167585
The function was executed in 0.10041518211364746 seconds


In [38]:
import numpy as np
import time

# Read data 
data = initial_data

# List to store execution times
number_of_executions = 10
execution_times = []

# Smoothing factors
alpha = 0.7
beta = 0.3
gamma = 0
season_length = 0  # Season = 1 Day for 1 value per hour

# Holt-Winters
for i in range(number_of_executions):

    # Start teh timer
    start_time = time.time()

    # Holt-Winters
    triple_exponential_smoothing_basic(data, alpha, beta, gamma, season_length)

    # Stop the timer
    end_time = time.time()
    function_time = end_time - start_time
    execution_times.append(function_time)
    #print(function_time)

# Calculate the function time
function_time = np.mean(execution_times)

# Print results
print("\n### Dataset energy_generation_solar")
print("### Holt-Winters\n")
print("The last smoothed value for energy_generation_solar is:", smoothed_value)
print(f"The function was executed in {function_time} seconds")


### Dataset energy_generation_solar
### Holt-Winters

The last smoothed value for energy_generation_solar is: 56.202351904167585
The function was executed in 0.07077980041503906 seconds


___
___
## Notes & Playground

- Doppeltes Exponential Smoothing:

In [80]:
import numpy as np
import time

# Heart Rate Data
heart_rate_data = initial_data

# Smoothing factors
alpha = 0.3
beta = 0.3

# List to store execution times
number_of_executions = 10
execution_times = []

# Perform double exponential smoothing on the data
for i in range(number_of_executions):
    start_time = time.time()

    n = heart_rate_data.shape[0]
    weights = alpha * (1 - alpha) ** np.arange(n)[::-1]
    smoothed_level = np.cumsum(weights * heart_rate_data)[::-1] / weights.sum()

    trend = np.zeros_like(heart_rate_data)
    trend[1:] = beta * (smoothed_level[1:] - smoothed_level[:-1]) + (1 - beta) * trend[:-1]

    smoothed_value = smoothed_level + trend

    # Stop the timer
    end_time = time.time()
    execution_times.append(end_time - start_time)

# Calculate the function time
function_time = np.median(execution_times)

# Print the results
print("### Dataset heart_rate_data")
print("### #2 Double Exponential Smoothing with Cumsum \n")
print("The last smoothed value for heart_rate_data is:", smoothed_value[-1])
print(f"The function was executed in {function_time} seconds")


### Dataset heart_rate_data
### #2 Double Exponential Smoothing with Cumsum 

The last smoothed value for heart_rate_data is: 0.0
The function was executed in 0.0021077394485473633 seconds


In [2]:
import numpy as np
import time

# Heart Rate Data
heart_rate_data = initial_data

# Smoothing factors
alpha = 0.7
beta = 0.3

# List to store execution times
number_of_executions = 10
execution_times = []

# Perform double exponential smoothing on the data
for i in range(number_of_executions):
    start_time = time.time()

    n = heart_rate_data.shape[0]
    weights = alpha * (1 - alpha) ** np.arange(n)[::-1]
    smoothed_level = np.cumsum(weights * heart_rate_data)[::-1] / weights.sum()

    trend_weights = beta * (1 - beta) ** np.arange(n-1)[::-1]
    smoothed_trend = np.cumsum(trend_weights * (smoothed_level[1:] - smoothed_level[:-1]))[::-1] / trend_weights.sum()

    smoothed_value = smoothed_level + np.concatenate(([0], smoothed_trend))

    # Stop the timer
    end_time = time.time()
    execution_times.append(end_time - start_time)

# Calculate the function time
function_time = np.median(execution_times)

# Print the results
print("### Dataset heart_rate_data")
print("### #2 Double Exponential Smoothing with Cumsum for Level and Trend \n")
print("The last smoothed value for heart_rate_data is:", smoothed_value[-1])
print(f"The function was executed in {function_time} seconds")


### Dataset heart_rate_data
### #2 Double Exponential Smoothing with Cumsum for Level and Trend 

The last smoothed value for heart_rate_data is: 0.0
The function was executed in 0.0014590024948120117 seconds


In [29]:
import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Example dataset
data = np.genfromtxt('/Users/niklas/Documents/GitHub/Uni/10_Masterarbeit/data_files/prepocessed_datafiles/energy_generation_solar_small.csv', delimiter=',', skip_header=1)

# Create a Pandas Series
series = pd.Series(data)

# Fit the model
model = ExponentialSmoothing(series, trend='add', seasonal='add', seasonal_periods=24).fit()

# Get the fitted values (smoothed values)
fitted_values = model.fittedvalues

# Get the last smoothed value
last_smoothed_value = fitted_values.iloc[-1]

# Print the last smoothed value
print("The last smoothed value for wind_speed is:", last_smoothed_value)

/Users/niklas/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/holtwinters/model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


The last smoothed value for wind_speed is: -134.59474060105475


___
___
## ARCHIVE

In [ ]:
import numpy as np
import time

# Read data y
# data = np.array([10, 12, 13, 12, 11, 14, 15, 16, 15, 14])
data = np.genfromtxt('/Users/niklas/Documents/GitHub/Uni/10_Masterarbeit/data_files/prepocessed_datafiles/wind_speed_small.csv', delimiter=',', skip_header=1)

# Smoothing factors
alpha = 0.2
beta = 0.1
gamma = 0.3
season_length = 24 * 365  # 1 Year for 1 value per hour

# Initialization of parameter arrays
n = len(data)
level = np.zeros(n)
trend = np.zeros(n)
season = np.zeros(n)
smoothed = np.zeros(n)

# Assign initial values
level[0] = data[0]
trend[0] = data[1] - data[0] if n > 1 else 0
season[:season_length] = data[:season_length] - level[0] if n >= season_length else 0

# Vectorized Holt-Winters
indices = np.arange(n)
season_indices = (indices - season_length) % season_length

# Level
level[1:] = alpha * (data[1:] - season[season_indices[1:]]) + (1 - alpha) * (level[:-1] + trend[:-1])

# Trend
trend[1:] = beta * (level[1:] - level[:-1]) + (1 - beta) * trend[:-1]

# Season
season[season_length:] = gamma * (data[season_length:] - level[season_length - 1:-1]) + (1 - gamma) * season[:n - season_length]

# Smoothed values
smoothed = level + trend + season[season_indices]

# Print results
print("### Dataset energy_generation_solar")
print("### Holt-Winters\n")
print("The last smoothed value for energy_generation_solar is:", smoothed[-1])
